# Course 4 Module 2 Lesson 3 Screencasts

## M2L3SC1: Fine-Tuning a Transformer with Hugging Face

### Step 1: Setting Up the Environment


In [ ]:
!pip install transformers torch --quiet
!pip install datasets==2.14.5 --quiet
!pip install fsspec==2025.3.0 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 10.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not 

### Step 2: Loading Pre-Trained Model

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

### Step 3: Setting up Fine-Tuning

In [ ]:
# Minimal DSPy classification example.

!pip install dspy>=2.6.0
import os, random, dspy

# 1) ------------- Configure your LM ------------------
# put your key in the environment first:
api_key = "add-api-key-here"
lm = dspy.LM("openai/gpt-3.5-turbo", max_tokens=256, api_key=api_key)
dspy.configure(lm=lm)

# 2) ------------- Define labels & module -------------
CLASSES = ["positive", "negative"]

# Declarative signature string:  text -> label ∈ Literal{...}
classify = dspy.Predict(f"text -> label:Literal{CLASSES}")  # :contentReference[oaicite:1]{index=1}

# 3) ------------- Tiny training set ------------------
seed_examples = [
    ("I absolutely loved this movie, highly recommend!",  "positive"),
    ("Worst customer service I’ve ever experienced.",     "negative"),
    ("The product exceeded my expectations.",             "positive"),
    ("Totally disappointed – it broke on day one.",       "negative"),
    ("Fantastic flavour and quick delivery!",             "positive"),
    ("It’s not worth the money at all.",                  "negative"),
    ("Great value for the price.",                        "positive"),
    ("Terrible packaging, arrived damaged.",              "negative"),
]

trainset = [
    dspy.Example(text=t, label=l).with_inputs("text")
    for t, l in seed_examples
]

# 4) ------------- Compile with BootstrapFewShot ------
metric = lambda ex, pred, trace=None: int(pred.label == ex.label)          # 1 = correct, 0 = wrong
optimized_classifier = (
    dspy.BootstrapFewShot(metric=metric, max_labeled_demos=8, max_rounds=2).compile(classify, trainset=trainset)
)

# 5) ------------- Try it out --------------------------
tests = [
    "This laptop is fantastic for the price.",
    "The meal was cold and tasteless.",
]
for s in tests:
    pred = optimized_classifier(text=s)
    print(f"{s:<55} → {pred.label}")

  0%|          | 0/8 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [input_value=Message(content='[[ ## la...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
 50%|█████     | 4/8 [00:04<00:04,  1.24s/it]


Bootstrapped 4 full traces after 4 examples for up to 2 rounds, amounting to 4 attempts.
This laptop is fantastic for the price.                 → positive
The meal was cold and tasteless.                        → negative


## M2L3SC2: Understanding PEFT and Reinforcement Learning Fine-Tuning

### Step 1: LoRA Implementation

In [ ]:
!pip install peft --quiet

import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from peft import get_peft_model, LoraConfig, TaskType

model_name = "t5-base"

# Load model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Configure LoRA
lora_config = LoraConfig(
    r=2,
    lora_alpha=0.5,
    task_type=TaskType.SEQ_2_SEQ_LM,
    lora_dropout=0.1,
)

# Apply LoRA to model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

trainable params: 221,184 || all params: 223,124,736 || trainable%: 0.0991


### Step 2: Integrating Human Feedback

In [ ]:
!pip install trl --quiet

from transformers import AutoTokenizer, pipeline
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead, create_reference_model
from datasets import Dataset
import torch

# Load tokenizer and model
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Value + policy model
model = AutoModelForCausalLMWithValueHead.from_pretrained(model_name)
ref_model = create_reference_model(model)

# PPO config
ppo_config = PPOConfig(batch_size=2, mini_batch_size=1, learning_rate=1e-5)

# Reward model
reward_pipe = pipeline("text-classification", model="lvwerra/distilbert-imdb")

# Wrap reward function as required (dummy wrapper)
class RewardModel(torch.nn.Module):
    def forward(self, queries, responses, **kwargs):
        texts = [q + r for q, r in zip(queries, responses)]
        rewards = [torch.tensor(r["score"]) for r in reward_pipe(texts)]
        return rewards

reward_model = RewardModel()

# Dummy train dataset in HuggingFace format
data = {"query": ["Why is the sky blue?", "Tell me a joke."]}
train_dataset = Dataset.from_dict(data)

# PPOTrainer: 7 required positional args
trainer = PPOTrainer(
    ppo_config,          # args
    tokenizer,           # processing_class
    model,               # model
    ref_model,           # ref_model
    reward_model,        # reward_model (positional!)
    train_dataset,       # train_dataset
    model                # value_model (same as model here)
)

# PPO loop
for batch in trainer.dataloader:
    query_tensors = batch["input_ids"]
    response_tensors = trainer.generate(query_tensors, max_new_tokens=20, pad_token_id=tokenizer.eos_token_id)
    rewards = reward_model(batch["query"], [tokenizer.decode(r, skip_special_tokens=True) for r in response_tensors])
    stats = trainer.step(query_tensors, response_tensors, rewards)
    trainer.log_stats(stats, batch, rewards)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.4/366.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


AttributeError: 'AutoModelForCausalLMWithValueHead' object has no attribute 'generation_config'

### Step 3: Visualizing Results

In [ ]:
### Jeremy - This was suggested by the LLM, if it's possible to include that would be fantastic.
import matplotlib.pyplot as plt
import numpy as np

def display_images(original, generated):
    fig, axes = plt.subplots(1, 2, figsize=(8, 4))

    axes[0].imshow(np.clip(original, 0, 1))
    axes[0].set_title("Original Image")

    axes[1].imshow(np.clip(generated, 0, 1))
    axes[1].set_title("Generated Image")

    for ax in axes:
        ax.axis('off')

    plt.show()

# Example original and generated images
original_image = np.random.rand(64, 64, 3)  # Random image for demonstration
generated_image = np.zeros((64, 64, 3))  # Placeholder black image

# Display the images side by side
display_images(original_image, generated_image)